In [1]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive') 
#capire qua come fare prima di renderlo pubblico. 
#questa cella collega tutte le cartelle drive al notebook ma in realtà serve solo per accedere al file csv
#del democracy index

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
req = r.get('https://ec.europa.eu/trustfundforafrica/navigation/all-projects_en') #pagina con tutti i progetti attivi dell'EUTF

In [4]:
req

<Response [200]>

In [5]:
soup = BeautifulSoup(req.content,'html.parser')

In [6]:
elementi = soup.find_all(attrs={'class':'country__name'}) #trovo tutti gli elementi della pagina
                                                          #html che hanno come classe "country name"
nomi = list(map(lambda x: x.text,elementi)) #applico la funzione .text a ogni membro di elementi (così estraggo il testo come stringa)

In [7]:
print(len(nomi))
nomi[0]
#serve levare a capi e spazi bianchi

186


'\n            Tchad          '

In [8]:
nomi = list(map(str.strip,nomi)) #voilà

In [9]:
nomi_unici = pd.Series(nomi).unique() #creiamo una serie di pandas dalla lista e togliamo i doppioni

In [10]:
nomi_unici = pd.Series(nomi_unici) #pandas però è stupido e la funzione unique() ritorna un array e non una serie, 
                                  #quindi ricreiamo una serie di pandas dall'array perché ci serviranno funzioni particolari

In [11]:
df = pd.read_csv('/content/drive/MyDrive/articoli in lavorazione/EUTF/EIU- materiale Notebook/Democ-Index.csv',delimiter=',')

In [12]:
df.sample(4) 
#questo è il dataframe. 
#Nella cella seguente c'è la definizione dei vari regimi in base alle divisioni fatte dall'Economist stesso
#In realtà la funzione andava applicata al dataframe solo la prima volta che ho creato il file csv (quando non c'era ancora la terza colonna)
#ma vabbé

,Paese,Index,Regime
64,Guyana,6.25,Flawed democracy
53,Philippines,6.62,Flawed democracy
128,Kazakhstan,3.08,Authoritarian
18,Mauritius,8.08,Full democracy


In [13]:
#ci è servita solo la prima volta in realtà
def valuta(x):
    if x > 7.99:
        return 'Full democracy'
    elif x>5.99:
        return 'Flawed democracy'
    elif x>3.99:
        return 'Hybrid regime'
    else:
        return 'Authoritarian'

In [14]:
df['Regime'] = df['Index'].apply(valuta)

In [15]:
nomi_unici[~(nomi_unici.isin(df['Paese']))] 
#operatore ~ serve a invertire i valori booleani la serie
#controlliamo i Paesi citati nel sito dell'EUTF che NON SONO nell'elenco dell'Economist
#così puliamo i dati

0           Tchad
8      The Gambia
11    South Sudan
13     Mauritanie
15        Somalia
20         Guinée
21       Cameroun
dtype: object

In [16]:
#qui manualmente abbiamo sistemato tutto il possibile
nomi_unici[0]='Chad'
nomi_unici[8]='Gambia'
nomi_unici[13]='Mauritania'
nomi_unici[20]='Guinea'
nomi_unici[21] = 'Cameroon'

In [17]:
nuovo_nomi = nomi_unici 
#creiamo la variabile solo per chiarezza sintattica
#È la lista "pulita" dei Paesi in cui ci sono progetti dell'EUTF

In [18]:
vecchi_nomi = pd.Series(nomi).unique() 
#È la lista non pulita dei Paesi in cui ci sono progetti dell'EUTF

In [19]:
diz = dict(zip(vecchi_nomi,nuovo_nomi))
#creiamo un dizionario che collega ogni nome del Paese così: nome non pulito ---> nome pulito

In [20]:
diz

{'Tchad': 'Chad',
 'Burkina Faso': 'Burkina Faso',
 "Côte d'Ivoire": "Côte d'Ivoire",
 'Mali': 'Mali',
 'Morocco': 'Morocco',
 'Niger': 'Niger',
 'Djibouti': 'Djibouti',
 'Ghana': 'Ghana',
 'The Gambia': 'Gambia',
 'Ethiopia': 'Ethiopia',
 'Sudan': 'Sudan',
 'South Sudan': 'South Sudan',
 'Kenya': 'Kenya',
 'Mauritanie': 'Mauritania',
 'Senegal': 'Senegal',
 'Somalia': 'Somalia',
 'Nigeria': 'Nigeria',
 'Egypt': 'Egypt',
 'Tunisia': 'Tunisia',
 'Libya': 'Libya',
 'Guinée': 'Guinea',
 'Cameroun': 'Cameroon',
 'Eritrea': 'Eritrea',
 'Uganda': 'Uganda'}

In [21]:
nomi_pulito = list(map(lambda x: diz[x],nomi))  
#ricordiamo che "nomi" è la lista di tutti i Paesi in cui ci sono progetti dell'EUTF
#è una lista che contiene anche doppioni, perché a volte ci sono più progetti dell'EUTF nello stesso Paese
#in questo modo puliamo questa lista usando il dizionario

In [22]:
lista_regimi =[]
ndf = df.set_index('Paese')
for country in nomi_pulito:
    if country in ndf.index: #alcuni Paesi in cui ci sono progetti dell'EUTF non sono stati analizzati dall'Economist
        regime = ndf.loc[country]['Regime'] 
    else:
        print(country) #solo per controllare eventuali errori
        regime = 'NC'
    lista_regimi.append(regime)

#alla fine del ciclo abbiamo una lista che, per ogni progetto dell'EUTF, segna il tipo di regime del Paese

South Sudan
Somalia
South Sudan
Somalia
South Sudan
South Sudan
Somalia
South Sudan
Somalia
South Sudan
Somalia
Somalia
Somalia
Somalia
South Sudan
South Sudan
South Sudan
South Sudan
South Sudan
South Sudan
South Sudan


In [23]:
lista_finale = pd.Series(lista_regimi).value_counts()

In [24]:
lista_finale #113 progetti dell'EUTF si trovano in regimi autoritari

Authoritarian       113
Hybrid regime        50
NC                   21
Flawed democracy      2
dtype: int64

In [25]:
pr = list(zip(nomi_pulito,lista_regimi))

In [26]:
pd.Series(pr).unique()

array([('Chad', 'Authoritarian'), ('Burkina Faso', 'Authoritarian'),
       ("Côte d'Ivoire", 'Hybrid regime'), ('Mali', 'Authoritarian'),
       ('Morocco', 'Hybrid regime'), ('Niger', 'Authoritarian'),
       ('Djibouti', 'Authoritarian'), ('Ghana', 'Flawed democracy'),
       ('Gambia', 'Hybrid regime'), ('Ethiopia', 'Authoritarian'),
       ('Sudan', 'Authoritarian'), ('South Sudan', 'NC'),
       ('Kenya', 'Hybrid regime'), ('Mauritania', 'Hybrid regime'),
       ('Senegal', 'Hybrid regime'), ('Somalia', 'NC'),
       ('Nigeria', 'Hybrid regime'), ('Egypt', 'Authoritarian'),
       ('Tunisia', 'Hybrid regime'), ('Libya', 'Authoritarian'),
       ('Guinea', 'Authoritarian'), ('Cameroon', 'Authoritarian'),
       ('Eritrea', 'Authoritarian'), ('Uganda', 'Hybrid regime')],
      dtype=object)

In [28]:
def serializza(serie):
    return pd.Series(serie.unique())

In [29]:
pd.Series.unico = serializza

In [33]:
pd.Series(pr).unico().apply(lambda x: x[1]).value_counts()

Authoritarian       12
Hybrid regime        9
NC                   2
Flawed democracy     1
dtype: int64